Ошибки посмотреть под ячейками  
На веб морде эйрфлоу предупреждение: "The scheduler does not appear to be running. The DAGs list may not update, and new tasks will not be scheduled".  
Ворнинги:
- The virtalenv_python example task requires virtualenv, please install it
- Install Kubernetes dependencies with: pip install apache-airflow[cncf.kubernetes]

Код из статей:
- [Как запустить Apache AirFlow в Google Colab с ngrok](https://bigdataschool.ru/blog/how-to-launch-airflow-on-colab-with-ngrok.html)
- [Как запустить DAG AirFlow в Google Colab: простой пример](https://bigdataschool.ru/blog/how-to-run-user-dag-airflow-in-colab-with-ngrok.html) (перепроверить работу начала второй части и продолжить с Запуск пользовательского DAG)

In [1]:
#############################ячейка №1 в Google Colab###########################
#Установка Apache Airflow и инициализация базы данных
!pip install apache-airflow
!airflow db init
## initdb - было так в оригинале, но пишет об ошибке и замене команды
#Установка инструмента ngrok для создания безопасного туннеля
#для доступа к веб-интерфейсу Airflow из любого места
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install pyngrok
#импорт модулей
from pyngrok import ngrok
import sys
import os
import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/airflow/dags', exist_ok=True)
dags_folder = os.path.join(os.path.expanduser("~"), "airflow", "dags")
os.makedirs(dags_folder, exist_ok=True)

DB: sqlite:////root/airflow/airflow.db
[2023-12-14T09:42:10.486+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-12-14T09:42:10.488+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-12-14T09:42:11.190+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-12-14T09:42:11.191+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-12-14T09:42:11.193+0000] {db.py:1620} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [unusual_prefix_e1e1e746b8b87627f53519aca03f086299788fc2_tutorial_taskflow_api_virtualenv] The tutorial_taskflow_api_virtualenv example DAG requires virtualenv, please install it.
WARNI [unusual_prefix_de310399792f7c66d0607289cf7f70223c00f7ca_example_python_operator] The virtalenv_python example task requires virtualenv, please install it.
WARNI [unusual_prefix_4c1c0063a36d033a56d4343efc723c3a45c1bd59_example

In [2]:
#######################ячейка №2 в Google Colab########################
#Задание переменной auth_token для аутентификации в сервисе ngrok.
auth_token = "2WhKx82LS1maMJ2dMagXgh9ipsb_3fcJGdvLkWMHihaHRpLju" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel
# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar
# and copy the API key
#Аутентификация в сервисе ngrok с помощью auth_token
os.system(f"ngrok authtoken {auth_token}")
#Запуск ngrok, который создаст публичный URL для сервера через туннель
#для доступа к веб-интерфейсу Airflow из любого места.
#addr="8888" указывает на порт, на котором запущен веб-сервер Airflow,
#а proto="http" указывает на использование протокола HTTP
public_url = ngrok.connect(addr="8888", proto="http")
#Вывод публичного URL для доступа к веб-интерфейсу Airflow
print("Адрес Airflow GUI:", public_url)

[2023-12-14T09:43:10.579+0000] {ngrok.py:276} INFO - Opening tunnel named: http-8888-b73f33b3-9499-47b0-bda3-ce953eb138c7
[2023-12-14T09:43:10.624+0000] {process.py:99} INFO - t=2023-12-14T09:43:10+0000 lvl=info msg="no configuration paths supplied"
[2023-12-14T09:43:10.632+0000] {process.py:99} INFO - t=2023-12-14T09:43:10+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
[2023-12-14T09:43:10.635+0000] {process.py:99} INFO - t=2023-12-14T09:43:10+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
[2023-12-14T09:43:10.682+0000] {process.py:99} INFO - t=2023-12-14T09:43:10+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
[2023-12-14T09:43:11.012+0000] {process.py:99} INFO - t=2023-12-14T09:43:11+0000 lvl=info msg="client session established" obj=tunnels.session obj=csess id=5a898603635d
[2023-12-14T09:43:11.015+0000] {process.py:99} INFO - t=2023-12-14T09:43:11+0000 lvl=info m

In [ ]:
## выше адрес вида https://2890-34-69-125-71.ngrok-free.app,
## по которому  зайдем в веб-интерфейс Apache AirFlow

In [3]:
#############################ячейка №3 в Google Colab#########################
!airflow db init #Инициализация базы данных Airflow  ## это же было в первой ячейке
!airflow db upgrade #Обновление базы данных Airflow
## !airflow upgradedb - в оригинале было, но ворнинги требуют новый синтаксис
#Создание нового пользователя в Apache Airflow с именем пользователя anna, именем Anna, фамилией Anna, адресом электронной почты anna.doe@example.com и паролем password.
#Этот пользователь будет иметь роль Admin, которая дает полный доступ к интерфейсу Airflow.
!airflow users create --username anna --firstname Anna --lastname Anna --email anna.doe@example.com --role Admin --password password

DB: sqlite:////root/airflow/airflow.db
[2023-12-14T09:43:25.318+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-12-14T09:43:25.319+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-12-14T09:43:26.156+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-12-14T09:43:26.157+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-12-14T09:43:26.160+0000] {db.py:1620} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
WARNI [unusual_prefix_e1e1e746b8b87627f53519aca03f086299788fc2_tutorial_taskflow_api_virtualenv] The tutorial_taskflow_api_virtualenv example DAG requires virtualenv, please install it.
WARNI [unusual_prefix_de310399792f7c66d0607289cf7f70223c00f7ca_example_python_operator] The virtalenv_python example task requires virtualenv, please install it.
WARNI [unusual_prefix_4c1c0063a36d033a56d4343efc723c3a45c1bd59_example

In [4]:
##########################ячейка №5 в Google Colab#######################
#запуск веб-сервера Apache Airflow на порту 8888. Веб-сервер Airflow предоставляет пользовательский интерфейс
#для управления DAG’ами,
#просмотра логов выполнения задач, мониторинга прогресса выполнения
!airflow webserver --port 8888

  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:8888
Timeout: 120
Logfiles: - -
Access Logformat: 
/usr/local/lib/python3.10/dist-packages/flask_limiter/extension.py:336 UserWarning: Using the in-memory storage for tracking rate limits as no storage was explicitly specified. This is not recommended for production use. See: https://flask-limiter.readthedocs.io#configuring-a-storage-backend for documentation about configuring the storage backend.
[2023-12-14 09:44:26 +0000] [9373] [INFO] Starting gunicorn 21.2.0
[2023-12-14 09:44:26 +0000] [9373] [INFO] Listening at: http://0.0.0.0:8888 (9373)
[2023-12-14 09:44:26 +0000] [9373] [INFO] Using worker: sync
[2023-12-14 09:44:26 +0000] [9395] [INFO] Booting worker with pid: 9395
[2023-12-14 09:4

In [ ]:
## Теперь можно войти в веб-интерфейс Apache AirFlow,
## по ссылке, ранее сгенерированной утилитой ngrok.
## Надо ввести логин и пароль пользователя, созданного ранее: anna, password

-------------

In [1]:
###############################################ячейка №1 в Google Colab#############################
#Установка Apache Airflow и инициализация базы данных.
!pip install apache-airflow
## !airflow init  ## еще будет в следующей ячейке
## initdb - было так в оригинале, но пишет об ошибке и замене команды
#Установка инструмента ngrok для создания безопасного туннеля для доступа к веб-интерфейсу Airflow из любого места
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install pyngrok

## !pip install sqlalchemy
## ругается установка на конфликт со старой версией алхимии
## но отдельная установка в лоб не проходит: уже установлено

#импорт модулей
from pyngrok import ngrok
import sys
import os
import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

## свой dag будем хранить на гугл-драйв
from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/airflow/dags', exist_ok=True)
dags_folder = os.path.join(os.path.expanduser("~"), "airflow", "dags")
os.makedirs(dags_folder, exist_ok=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.3/511.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 7.1 MB

In [2]:
###############################################ячейка №2 в Google Colab#############################
#Задание переменной auth_token для аутентификации в сервисе ngrok.
auth_token = "2WhKx82LS1maMJ2dMagXgh9ipsb_3fcJGdvLkWMHihaHRpLju"
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel
# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
#Аутентификация в сервисе ngrok с помощью auth_token
os.system(f"ngrok authtoken {auth_token}")
#Запуск ngrok, который создаст публичный URL для сервера через туннель
#для доступа к веб-интерфейсу Airflow из любого места.
#addr="8888" указывает на порт, на котором запущен веб-сервер Airflow, а proto="http" указывает на использование протокола HTTP
public_url = ngrok.connect(addr="8888", proto="http")
#Вывод публичного URL для доступа к веб-интерфейсу Airflow
print("Адрес Airflow GUI:", public_url)

[2023-12-14T09:26:22.229+0000] {ngrok.py:276} INFO - Opening tunnel named: http-8888-386811f8-e5b2-49bc-bc5a-c3f90605535a
[2023-12-14T09:26:22.304+0000] {process.py:99} INFO - t=2023-12-14T09:26:22+0000 lvl=info msg="no configuration paths supplied"
[2023-12-14T09:26:22.311+0000] {process.py:99} INFO - t=2023-12-14T09:26:22+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
[2023-12-14T09:26:22.316+0000] {process.py:99} INFO - t=2023-12-14T09:26:22+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
[2023-12-14T09:26:22.429+0000] {process.py:99} INFO - t=2023-12-14T09:26:22+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
[2023-12-14T09:26:22.757+0000] {process.py:99} INFO - t=2023-12-14T09:26:22+0000 lvl=info msg="client session established" obj=tunnels.session obj=csess id=bf9112e08de7
[2023-12-14T09:26:22.760+0000] {process.py:99} INFO - t=2023-12-14T09:26:22+0000 lvl=info m

In [ ]:
## ip для входа чуть выше в формате типа https://ec27-34-86-112-48.ngrok-free.app

In [3]:
#############################ячейка №3 в Google Colab#########################
!airflow db init #Инициализация базы данных Airflow
## это же было в первой ячейке
## но без этого не подгружаются примеры?
!airflow db upgrade #Обновление базы данных Airflow
## !airflow upgradedb - в оригинале было, но ворнинги требуют новый синтаксис
#Создание нового пользователя в Apache Airflow с именем пользователя anna, именем Anna, фамилией Anna, адресом электронной почты anna.doe@example.com и паролем password.
#Этот пользователь будет иметь роль Admin, которая дает полный доступ к интерфейсу Airflow.
!airflow users create --username anna --firstname Anna --lastname Anna --email anna@example.com --role Admin --password password

DB: sqlite:////root/airflow/airflow.db
Performing upgrade to the metadata database sqlite:////root/airflow/airflow.db
[2023-12-14T09:27:56.905+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-12-14T09:27:56.905+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
[2023-12-14T09:27:56.909+0000] {migration.py:213} INFO - Context impl SQLiteImpl.
[2023-12-14T09:27:56.909+0000] {migration.py:216} INFO - Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running stamp_revision  -> 405de8318b3a
Database migrating done!
/usr/local/lib/python3.10/dist-packages/flask_limiter/extension.py:336 UserWarning: Using the in-memory storage for tracking rate limits as no storage was explicitly specified. This is not recommended for production use. See: https://flask-limiter.readthedocs.io#configuring-a-storage-backend for documentat

In [ ]:
## копируем файл с дагом в airflow
!cp /content/airflow/dags/ANNA_DAG_SaveFile.py ~/airflow/dags/ANNA_DAG_SaveFile.py
## ?
!airflow dags unpause ANNA_DAG_SaveFile

In [ ]:
запуск веб-сервера Airflow на порту 8888 в фоновом режиме (daemon mode)
!airflow webserver --port 8888 --daemon

можно открыть веб-браузер и перейти на страницу URL-адреса, полученного ранее с помощью утилиты ngrok, которая перебрасывает локальный хост http://localhost:8888 на внешний URL-адрес. Теперь можно войти в веб-интерфейс Apache AirFlow, используя логин и пароль ранее созданного пользователя.

In [ ]:
## запустить планировщик AirFlow. Удобнее всего это сделать тоже в фоновом режиме с помощью аргумента —daemon
!airflow scheduler --daemon

In [ ]:
## По завершении работы с веб-интерфейсом AirFlow надо освободить занятый порт
!pkill -f ngrok #поиск и завершение процесса, который запущен с помощью команды ngrok
# отключение публичного URL, созданного с помощью ngrok для веб-сервера Airflow
ngrok.disconnect(public_url=public_url)